In [1]:
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize, wordpunct_tokenize
from nltk.corpus import stopwords

In [3]:
train_data = pd.read_csv("../Data/training.1600000.processed.noemoticon.csv", encoding="ISO-8859-1")

In [4]:
train_data.head(4)

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [5]:
train_data.columns = ['target', 'ids', 'date', 'flag', 'user', 'text']

In [6]:
train_data.columns

Index(['target', 'ids', 'date', 'flag', 'user', 'text'], dtype='object')

In [7]:
train_data.drop(columns=['ids', 'date', 'flag', 'user',], axis=1, inplace=True)

In [8]:
train_data.sample(3)

,target,text
1327521,4,@mag_nation I can see it too
1340762,4,Good night everyone!
1394706,4,@ScruffyPanther ;-) they are both bound to com...


In [9]:
train_data.target.nunique()

2

In [10]:
counter = 0
for i in range(len(train_data)):
    if train_data["target"][i] == 2:
        counter = counter + 1
        train_data["target"][i] = 4

In [11]:
train_data.target.nunique()

2

In [12]:
counter

0

In [13]:
train_data.target.replace(to_replace=4, value=1, inplace=True)

In [14]:
def processing(tweet):
    tweet.lower()
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))',' ',tweet)
    tweet = re.sub('@[^\s]+',' ', tweet)
    tweet = re.sub('[\s]+', ' ', tweet)
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
    return tweet

In [15]:
train_data['text'] = train_data['text'].apply(processing)

In [16]:
allowed_word_types = ["JJ", "JJR", "JJS", "NN", "NNS", "RB", "RBR", "RBS", "VB", "VBD", "VBG", "VBN", "VBP"]
stop_words = set(stopwords.words("English"))

In [17]:
def nltk_process(tweet):
    sentence = nltk.pos_tag(word_tokenize(str(tweet)))
    full = []
    for s in sentence:
        if s[1] in allowed_word_types and s[0] not in stop_words:
            full.append(s[0])
    fulls = " ".join(full)
    return fulls

In [18]:
train_data['tweets'] = train_data['text'].apply(nltk_process)

In [19]:
train_data.drop(columns=['text'], inplace=True)

In [20]:
train_data.dropna(axis=0, inplace=True)

In [23]:
train_data.to_csv("../Data/train_data.csv")